In [2]:
import pandas as pd
import os
import io

In [3]:
final_list=pd.read_excel("corpus-final09.xls",sheet_name="File list")

In [4]:
final_list

,File,Group,Person,Task,Category,Native English,Knowledge,Difficulty
0,g0pA_taska.txt,0,A,a,non,native,1,1
1,g0pA_taskb.txt,0,A,b,cut,native,4,3
2,g0pA_taskc.txt,0,A,c,light,native,5,3
3,g0pA_taskd.txt,0,A,d,heavy,native,3,4
4,g0pA_taske.txt,0,A,e,non,native,4,3
...,...,...,...,...,...,...,...,...
90,g4pE_taska.txt,4,E,a,heavy,non-native,1,2
91,g4pE_taskb.txt,4,E,b,light,non-native,3,2
92,g4pE_taskc.txt,4,E,c,cut,non-native,4,2
93,g4pE_taskd.txt,4,E,d,non,non-native,4,4


In [19]:
#load text files to dictionary
ls={}
directory=os.getcwd()
for i,filename in enumerate(os.listdir(directory)):
    if filename.endswith(".txt"):
#         print(filename)
        f = io.open(filename, mode="r", encoding="utf-8")
        lines = f.read()
        ls[filename.replace('.txt','')]=re.sub('[^a-zA-Z0-9]', ' ', lines.rstrip())

In [20]:
ls

{'g0pA_taska': 'Inheritance is a basic concept of Object Oriented Programming where the basic idea is to create new classes that add extra detail to existing classes  This is done by allowing the new classes to reuse the methods and variables of the existing classes and new methods and classes are added to specialise the new class  Inheritance models the  is kind of  relationship between entities  or objects   for example  postgraduates and undergraduates are both kinds of student  This kind of relationship can be visualised as a tree structure  where  student  would be the more general root node and both  postgraduate  and  undergraduate  would be more specialised extensions of the  student  node  or the child nodes   In this relationship  student  would be known as the superclass or parent class whereas   postgraduate  would be known as the subclass or child class because the  postgraduate  class extends the  student  class   Inheritance can occur on several layers  where if visualis

In [31]:
#Case Folding, rudimernary normalization and spliting string to words
import re
from nltk.corpus import stopwords 
dict_list={}
stop_words = set(stopwords.words('english'))
for key in ls.keys():
    dict_list[key]=[]
    [dict_list[key].append(re.sub('[^ a-zA-Z0-9]', '',x)) for x in ls[key].lower().split(" ") if x not in stop_words] 
#     [dict_list[key].append(re.sub('[^ a-zA-Z0-9]', '',x)) for x in ls[key].lower().split(" ") if x not in dict_list[key] if x not in stop_words] #Remove duplicates also

In [32]:
len(dict_list['g0pA_taska']),len(ls['g0pA_taska'].split(" "))

(68, 271)

In [42]:
#Lemmatization
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer() 
dict_lemmatized={}
word_counter=0
for key in ls.keys():
    dict_lemmatized[key]=[]
    [dict_lemmatized[key].append(lemmatizer.lemmatize(x)) for x in dict_list[key]]
    word_counter+=len(dict_lemmatized[key])
# word_counter

8156

In [36]:
len(dict_lemmatized['g0pA_taska'])

68

In [38]:
all_words=[]
for key in ls.keys():
    [all_words.append(x) for x in dict_lemmatized[key] if x not in all_words]

In [40]:
len(all_words)

1769

In [44]:
inverted_index={}
for word in all_words:
    inverted_index[word]=[]
    [inverted_index[word].append(key) for key in ls.keys() if word in dict_lemmatized[key]]

In [47]:
inverted_index,len(inverted_index)

({'inheritance': ['g0pA_taska',
   'g0pB_taska',
   'g0pC_taska',
   'g0pD_taska',
   'g0pE_taska',
   'g1pA_taska',
   'g1pB_taska',
   'g1pD_taska',
   'g2pA_taska',
   'g2pB_taska',
   'g2pC_taska',
   'g2pE_taska',
   'g3pA_taska',
   'g3pB_taska',
   'g3pC_taska',
   'g4pB_taska',
   'g4pC_taska',
   'g4pD_taska',
   'g4pE_taska',
   'orig_taska'],
  'basic': ['g0pA_taska',
   'g0pB_taska',
   'g0pE_taskc',
   'g1pB_taska',
   'g2pE_taske',
   'g3pC_taskb',
   'g4pC_taskc',
   'g4pD_taska'],
  'concept': ['g0pA_taska',
   'g0pB_taska',
   'g0pB_taskb',
   'g0pE_taska',
   'g1pB_taska',
   'g2pE_taska',
   'g3pB_taska',
   'g4pC_taska',
   'g4pD_taska',
   'g4pD_taske',
   'g4pE_taskb',
   'g4pE_taske',
   'orig_taska',
   'orig_taskb'],
  'object': ['g0pA_taska',
   'g0pA_taskc',
   'g0pB_taska',
   'g0pB_taskc',
   'g0pC_taska',
   'g0pD_taska',
   'g0pD_taskc',
   'g0pE_taska',
   'g1pA_taska',
   'g1pA_taskc',
   'g1pA_taske',
   'g1pB_taska',
   'g1pB_taskc',
   'g1pD_taska',


In [48]:
lemmatizer.lemmatize('headset'),lemmatizer.lemmatize('headphones')

('headset', 'headphone')

In [49]:
#testing out stemming
from nltk.stem import PorterStemmer
stemmer= PorterStemmer()
stemmer.stem('headset'),stemmer.stem('headphones')

('headset', 'headphon')

In [ ]:
### TO-DO #####
### Implement similarity score